### Get Input File & Setup

In [1]:
table=[
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,1],
    [0,1,1,1,1],
    [0,1,1,0,1],
    [1,0,1,0,1],
    [0,0,0,1,1],
    [0,0,0,0,0]
]

B = {
    'A','B','C','D','E'
}

FactorHash = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4
}

NegFactorSet = {}

### Perform Alg on File

In [2]:
#STEP 0

# FactorTable is a hashtable/dictionary of all factors, factorA -> index | key -> value
# For everything in W, we will have to test this
# Over every factor, we are collecting the set of factors that are possible effects, this set is W
# B: set of all factors, main_table: C
def step0(main_table, B, FactorTable, NegFactorSet):
    print("mainTable:")
    print(main_table)
    W = []
    for factor in B:
        print("Looking at " + factor)
        #(2)
        if rowDuplicity(main_table, FactorTable[factor]):
            print(factor + " failed row duplicity")
            continue
        #(3)
        elif isNeg(FactorTable[factor], NegFactorSet):
            print(factor + " failed isNeg")
            continue
        W.append(factor)
    return W


import copy
'''
    Check if the same row but negative factor already exists, if not, insert into set, else return 1
    int[i][j] table, i= coincidence, j = factor index
'''
def rowDuplicity(table, factor):
    rowSet = []
    #loop through every row in table
    for coincidence in table:
        modif_coincidence = copy.deepcopy(coincidence)
        modif_coincidence[factor] = 1-modif_coincidence[factor]
        if modif_coincidence in rowSet:
            return 1
        rowSet.append(coincidence)
        
def isNeg(factor, NegFactorSet):
    if factor in NegFactorSet:
        return 1
    return 0

In [3]:
potential_effects = step0(table, B, FactorHash, NegFactorSet)

mainTable:
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
Looking at D
D failed row duplicity
Looking at E
Looking at C
Looking at B
B failed row duplicity
Looking at A
A failed row duplicity


### Main Loop

In [4]:
#-- HELPER FUNCTIONS - SELF SET FUNCTIONS --##

def setIn(fauxSet, coincidence):
    #loop through fauxSet
#     print("---------")
#     print(coincidence)
#     print(fauxSet)
#     print("---------")
    for elem in fauxSet:
        if len(elem) == len(coincidence):
#             print("888888")
#             print(elem)
#             print(coincidence)
#             print("888888")
            for index, fac in enumerate(elem):
                #compare each index in fauxElement to coincidence
                if coincidence[index] != fac:
                    break
                if index == len(elem)-1:
                    return True
    return False

def pairListsEqual(pairList1, pairList2):
    for pair1 in pairList1:
        exists = False
        for pair2 in pairList2:
            if pair1[0] == pair2[0] and pair1[1] == pair2[1]:
                exists = True
        if not exists:
            return False
    return True

def pairListIn(conditionList, pairList):
    for elem in conditionList:
        if pairListsEqual(elem, pairList):
            return True
    return False

#------------------------

def pairListListsEqual(pairListList1, pairListList2):
    if len(pairListList1) == len(pairListList2):
        for pairList in pairListList1:
            if not pairListIn(pairListList2, pairList):
                return False
        return True
    return False

def pairListListIn(conditionList, pairListList):
    for elem in conditionList:
        if pairListListsEqual(elem, pairListList):
            return True
    return False
        

In [5]:

#-------------------------------------------------------------------------------
#STEP 1
#FOR LOOP
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], 
#  [0, 0, 0, 0, 0]]
# C
# []
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
# E
# []
#--------------------------------
#STEP 2

from numpy import *

'''
table: int[][] : double array of 1/0
factor: int : index of Z_i 
coincidence: X_k Z_i
modif_coincidence: X_k !Z_i
returns set of rows where coincidence X_k is sufficient for Z_i
'''
def step2(main_table, factor):
    pairListList = []
    for coincidence in main_table:
        pairList = []
        for index,value in enumerate(coincidence):
            if index != FactorHash[factor]:
                pairList.append((index,value))
            
        if check_sufficient(pairList, main_table, factor):
            pairListList.append(pairList)
    return pairListList
#--------------------------------
#STEP 3

from itertools import permutations
import copy
'''

'''
def step3(rowSet, main_table, effect):
    minimally_sufficient_conditions = []
    
    for pairList in rowSet:
        prm_list = list(permutations(range(0,len(pairList))))
        for prm in prm_list:
            #create modif_pairList
            modif_pairList = []
            for index in prm:
                modif_pairList.append(pairList[index])
            tmp_list = copy.deepcopy(modif_pairList)
            for factor in tmp_list:
                #remove
                modif_pairList.remove(factor)
                #is sufficient?
                if not check_sufficient(modif_pairList, table, effect):
                    modif_pairList.append(factor)
            if not pairListIn(minimally_sufficient_conditions,modif_pairList):
                minimally_sufficient_conditions.append(modif_pairList)
                    
    return minimally_sufficient_conditions
                    
# returns 1 if sufficient
def check_sufficient(pairList, table, effect):
    foundOne = False
    for row in table:
        tmpvar = True
        for pair in pairList:
            if pair[1] != row[pair[0]]:
                tmpvar = False
                break
        if tmpvar:
            foundOne = True
            if row[FactorHash[effect]] == 0:
                return 0
    return foundOne



In [6]:
#--------------------------------
#STEP 4
#END OF FOR LOOP -- WE ARE IGNORING THIS STEP FOR EFFICIENCY

#--------------------------------
#STEP 5

def step5(main_table, sufficient_pair_lists, effect):
    return check_necessary(main_table, sufficient_pair_lists, effect)

#--------------------------------
#STEP 6

from itertools import permutations

def step6(main_table, necessary_condition, effect):
    minimally_necessary_conditions = []
    prm_list = list(permutations(range(0, len(necessary_condition))))
    for prm in prm_list:
        test_necessary_condition = []
        for index in prm:
            test_necessary_condition.append(necessary_condition[index])
        for i in range(0, len(test_necessary_condition)):
            suff_cond = test_necessary_condition.pop(0)
            if not check_necessary(main_table, test_necessary_condition, effect):
                test_necessary_condition.append(suff_cond)
        if not pairListListIn(minimally_necessary_conditions, test_necessary_condition):
            minimally_necessary_conditions.append(test_necessary_condition)
    return minimally_necessary_conditions

#pair[0] is index, pair[1] is value        
def check_necessary(main_table, pair_lists, effect):
    for row in main_table:
        if row[FactorHash[effect]] == 1:
            necessary = False
            for pair_list in pair_lists:
                found_match = True
                for pair in pair_list:
                    if not (pair[1] == row[pair[0]]):
                        found_match = False
                if found_match:
                    necessary = True
                    break
            if not necessary:
                return False
    return True

In [15]:
print(table)
for effect in potential_effects:
    print("_________________" + effect + "____________________\n")
    
    print("Rowset_________________")
    rowSet = step2(table,effect)
    for pairList in rowSet:
        print(pairList)
    print("end Rowset_______________________\n")
    sufficientSet = step3(rowSet,table,effect)
    
    print("Sufficientset_________________")
    for pairList in sufficientSet:
        print(pairList)
        
    print("end Sufficientset_______________________\n")
        
    if step5(table, sufficientSet, effect):
        necessarySet = step6(table, sufficientSet, effect)
        print("Necessaryset_________________")
        for necessary in necessarySet:
            print(necessary)
        print("end Necessaryset_______________________\n")

[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 1], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
_________________E____________________

Rowset_________________
[(0, 1), (1, 1), (2, 1), (3, 1)]
[(0, 1), (1, 1), (2, 1), (3, 0)]
[(0, 1), (1, 0), (2, 1), (3, 1)]
[(0, 0), (1, 1), (2, 1), (3, 1)]
[(0, 0), (1, 1), (2, 1), (3, 0)]
[(0, 1), (1, 0), (2, 1), (3, 0)]
[(0, 0), (1, 0), (2, 0), (3, 1)]
end Rowset_______________________

Sufficientset_________________
[(3, 1)]
[(2, 1)]
[(1, 1)]
[(0, 1)]
end Sufficientset_______________________

Necessaryset_________________
[[(3, 1)], [(1, 1)], [(0, 1)]]
[[(3, 1)], [(2, 1)]]
end Necessaryset_______________________

_________________C____________________

Rowset_________________
[(0, 1), (1, 1), (3, 1), (4, 1)]
[(0, 1), (1, 1), (3, 0), (4, 1)]
[(0, 1), (1, 0), (3, 1), (4, 1)]
[(0, 0), (1, 1), (3, 1), (4, 1)]
[(0, 0), (1, 1), (3, 0), (4, 1)]
[(0, 1), (1, 0), (3, 0), (4, 1)]
end Rowset_______________________

Suff

### Framing